In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Model download from Hugging Face

In [1]:
# @title Install Vertex AI SDK and other required packages
%pip install --upgrade --user --quiet google-cloud-aiplatform \
                                      huggingface_hub[hf_transfer] \
                                      transformers

In [2]:
# @title Define constants
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
BUCKET_URI = "gs://sllm_checkpoints"  # @param {type:"string"}
MODEL_DIR = "Llama-3.2-3B-Instruct"  # @param {type:"string"}
ARTIFACT_NAME = "llama3.1_3b_inst"  # @param {type:"string"}

In [3]:
# @title GCP Authentication

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)


In [4]:
# @title Authenticate your Hugging Face account
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y


## Download a model using command

In [5]:
# @title Create local dir.
LOCAL_DIR="model/llama3.1-3b-it"
! mkdir -p {LOCAL_DIR}

In [6]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}/{MODEL_DIR}

Creating gs://sllm_checkpoints/...


In [ ]:
# @title Download model from Hugging face

#MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
MODEL_NAME = "microsoft/Phi-3.5-MoE-instruct"

! huggingface-cli download {MODEL_NAME} --exclude "*.bin" "*.pth" "*.gguf" ".gitattributes" --local-dir {LOCAL_DIR}

In [ ]:
# @title Upload model files into GCS
! gsutil -o GSUtil:parallel_composite_upload_threshold=150M -m cp -e -r {LOCAL_DIR}/* {BUCKET_URI}/{MODEL_DIR}/{ARTIFACT_NAME}

Copying file://model/llama3.1-3b-it/generation_config.json [Content-Type=application/json]...
Copying file://model/llama3.1-3b-it/config.json [Content-Type=application/json]...
Copying file://model/llama3.1-3b-it/LICENSE.txt [Content-Type=text/plain]...
Copying file://model/llama3.1-3b-it/model.safetensors.index.json [Content-Type=application/json]...
Copying file://model/llama3.1-3b-it/original/params.json [Content-Type=application/json]...
Copying file://model/llama3.1-3b-it/model-00001-of-00002.safetensors [Content-Type=application/octet-stream]...
Copying file://model/llama3.1-3b-it/original/tokenizer.model [Content-Type=application/octet-stream]...
Copying file://model/llama3.1-3b-it/model-00002-of-00002.safetensors [Content-Type=application/octet-stream]...
Copying file://model/llama3.1-3b-it/README.md [Content-Type=text/markdown]...
Copying file://model/llama3.1-3b-it/original/orig_params.json [Content-Type=application/json]...
Copying file://model/llama3.1-3b-it/special_tokens_

## Download model using python code

In [ ]:
# @title Helper function to download a model and store it into GCS
from transformers import AutoModel, AutoTokenizer
from google.cloud import storage

def model_download(model_name:str,
                   save_path:str,
                   bucket_name:str
                   ):

    # Download model and tokenizer.
    model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    # Save the model into Cloud storage.
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    for file_name in os.listdir(save_path):
      blob = bucket.blob(f"{save_path}/{file_name}")
      with open(os.path.join(save_path, file_name), "rb") as f:
        blob.upload_from_file(f)

    return model, tokenizer

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
save_path = "model"
bucket_name = "sllm_0106"

model, tokenizer = model_download(model_name,save_path,bucket_name)
